# Differences between Rotten Tomatoes' Tomatometer and Audience score - filtered by genre.
Does the difference between critic score and audience score vary by genre. The project's idea is to mine the famous [Rotten Tomatoes](https://www.rottentomatoes.com) for the scores of movies and then look-up the genres in imdb.

Once I got the data I will store it in a MySQL database and query from there.

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# Defining the options for our browser
chrome_options = Options()
chrome_options.add_argument("--headless") # to not display the page

# Defining the drivers and the URL
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://www.rottentomatoes.com/browse/dvd-streaming-all/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
# Scraping Functions
def get_movie_count(webdriver):
    html = BeautifulSoup(webdriver.page_source, 'html.parser')
    init_movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[1]
    movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[-1]
    
    return int(init_movie_count), int(movie_count)

def click_on_show_more(webdriver):
    webdriver.find_element_by_css_selector('.mb-load-btn').click()
    # no returns

def scrape_all_movies(webdriver):
    '''
    The function takes the webdriver and return the whole page source for further work.
    '''
    init_count, total_count = get_movie_count(webdriver)
    
    while init_count < total_count:
        click_on_show_more(webdriver)
        init_count, total_count = get_movie_count(webdriver)
    
    return webdriver.page_source

In [ ]:
# Scraping the actual page - can take a lot of time... get a coffee.
html = scrape_all_movies(driver)

In [30]:
def get_all_movies(html):
    soup_page = BeautifulSoup(html, 'html.parser')
    movie_infos = soup_page.find_all(attrs={'class':'movie_info'})
    
    movies = []
    item = 0
    for info in movie_infos:
        movie = { 'movie': title:info.find(attrs={'class':'movieTitle'}).text,
                  'critic_score':info.find_all(attrs={'class':'tMeterScore'})[0].text,
                   'user_score':info.find_all(attrs={'class':'tMeterScore'})[1].text}
        movies.append(movie)
        
        item += 1
        if item > 3:
            break
        
     
    return movies

In [31]:
movie_info = get_all_movies(driver)

<class 'bs4.element.ResultSet'>
Avengers: Infinity War
83%
91%


In [13]:
type(movie_info)

NoneType